In [2]:
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.layers import Input
from keras.models import Model, load_model
import numpy as np
import cv2
from keras.applications.inception_v3 import preprocess_input
from keras.applications.inception_v3 import decode_predictions
from keras.preprocessing import image
import matplotlib.pyplot as plt
import os
import glob
from os import listdir
from os.path import isfile, join
from keras.applications.inception_v3 import InceptionV3

Using TensorFlow backend.


In [3]:
from keras.layers import Dense, Flatten, Dropout, ZeroPadding3D
from keras.layers.recurrent import LSTM
from keras.models import Sequential, load_model
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import TimeDistributed
from keras.layers.convolutional import (Conv2D, MaxPooling3D, Conv3D,
    MaxPooling2D)

In [4]:
# this could also be the output a different Keras model or layer
input_tensor = Input(shape=(125, 125,3))  # this assumes K.image_data_format() == 'channels_last'
model1 = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=True)
model1.layers.pop()
model1.outputs = [model1.layers[-1].output]
model1.output_layers = [model1.layers[-1]]
model1.layers[-1].outbound_nodes = []

In [5]:
X=[]
Y=[]
active_drowsy=['E://CAPP//Active//','E://CAPP//Drowzy//']
for a in active_drowsy:
    people=os.listdir(a+'Images')
    for b in people:
        frames = [f for f in listdir(a+"Images\\"+b) if isfile(join(a+"Images\\"+b, f))]
        file=[]
        for c in frames:
            img = cv2.imread(a+"Images\\"+b+"\\"+c)
            img=cv2.resize (img,(125,125))
            img = np.expand_dims(img, axis=0)
            prediction=model1.predict(img)
            prediction=np.squeeze(prediction)
            x_list = prediction.tolist()
            file.append(x_list)
        X.append(file)
        if(a=='E://CAPP//Active//'):
            Y.append(0)
        else:
            Y.append(1)
    

In [6]:
temp=np.asarray(X)

In [7]:
temp.shape

(4, 40, 2048)

In [8]:
tempy=np.asarray(Y)

In [9]:
tempy.shape

(4,)

In [10]:
np.save("E:/CAPP/X.npy",temp)

In [11]:
np.save("E:/CAPP/Y.npy",tempy)

In [8]:
model = Sequential()
model.add(LSTM(2048, return_sequences=True,input_shape=(40,2048),dropout=0.5))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.00005),
metrics = ['accuracy'])

In [ ]:
model.fit(temp,tempy)

Epoch 1/1
